In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable

In [2]:
train_dataset = datasets.MNIST(root="./data",
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

Processing...
Done!


In [3]:
test_dataset = datasets.MNIST(root="./data",
                              train=False,
                              transform=transforms.ToTensor(),
                              )

In [4]:
train_dataset.train_data.size()

torch.Size([60000, 28, 28])

In [6]:
train_dataset.train_labels.size()

torch.Size([60000])

In [7]:
test_dataset.test_data.size()

torch.Size([10000, 28, 28])

In [13]:
test_dataset.test_labels.size()

torch.Size([10000])

In [14]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [15]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [16]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 =nn.Linear(32*7*7, 10)
        
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)
        
        out = self.maxpool1(out)
        
        out = self.cnn2(out)
        out = self.relu2(out)
        
        out = self.maxpool2(out)
        
        out = self.fc1(out.view(out.size(0), -1))
        
        return out
        
        

In [17]:
model = CNNModel()

In [18]:
criterion = nn.CrossEntropyLoss()

In [19]:
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [22]:
print(model.parameters())

print(len(list(model.parameters())))

print(list(model.parameters())[0].size())

print(list(model.parameters())[1].size())

print(list(model.parameters())[2].size())

print(list(model.parameters())[3].size())

print(list(model.parameters())[4].size())

print(list(model.parameters())[5].size())

<generator object Module.parameters at 0x1141b61a8>
6
torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])


In [25]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            correct = 0
            total = 0
            
            for images, labels in test_loader:
                images = Variable(images)
                lables = Variable(labels)
                
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)
                
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
                
            accuracy = 100 * correct /total
            
            print("Iteration {}, loss: {}, accuracy: {}".format(iter, loss.data[0], accuracy))

Iteration 500, loss: 0.21524402499198914, accuracy: 93.74
Iteration 1000, loss: 0.17868663370609283, accuracy: 94.87
Iteration 1500, loss: 0.1637900173664093, accuracy: 95.67
Iteration 2000, loss: 0.09537270665168762, accuracy: 96.44
Iteration 2500, loss: 0.08778399229049683, accuracy: 96.88
Iteration 3000, loss: 0.222147598862648, accuracy: 97.31


In [26]:
torch.save(model.state_dict(), "cnn_model.pkl")